In [1]:
#Import Tweepy-Python Library for interacting with Twitter API
import tweepy

In [2]:
#Generate API_KEY, API_SECRET_KEY, ACCESS_TOKEN_KEY, ACCESS_TOKEN_SECRET_KEY from https://developer.twitter.com/
import keys
API_KEY = keys.api_key
API_SECRET_KEY = keys.api_secret_key
ACCESS_TOKEN_KEY = keys.access_token
ACCESS_TOKEN_SECRET_KEY = keys.access_token_secret

In [3]:
#Twitter API Authentication 
auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN_KEY, ACCESS_TOKEN_SECRET_KEY)
api = tweepy.API(auth, wait_on_rate_limit= True, wait_on_rate_limit_notify= True)
auth_boolean =  False 
try:
    api.verify_credentials()
    auth_boolean = True
    print('Authentication Done')
except:

    print('Error during Authentication')

Authentication Done


In [5]:
last_tweet_id = api.mentions_timeline()[-1].id #Reference to next entry point

In [6]:
counter = 0 #Number of replies by this TweepyBot

if auth_boolean:
    #Loop through all mentioned tweets
    for tweet in api.mentions_timeline(since_id = last_tweet_id):
        if counter == 0:
            last_tweet_id = tweet.id #Store entry tweet ID
        counter += 1
        
        try:
            mention_tweet_id, video_tweet_id, mention_username  = tweet.id, tweet.in_reply_to_status_id, tweet.user.screen_name
            #Extracting link of video 
            video_tweet = api.get_status(video_tweet_id, tweet_mode = 'extended')
            
            if video_tweet.is_quote_status:
                entities = video_tweet.quoted_status.extended_entities
            else:
                entities= video_tweet.extended_entities
                
            urls = entities['media'][0]['video_info']['variants']
            url_list= [ urls[i]['url'] for i in range(len(urls)) ]

            msg = "Hey @" + mention_username + " !! Checkout : \n" 
            #Check for mp4 videos
            for obj in urls:
                if obj['url'][-10:-7] == 'mp4':
                    #Extract URL
                    url = obj['url']
                    splited_url = url.split('/')
                    #Extract Video Quality
                    video_quality = splited_url[-2]
                    #Add video quality and url to msg
                    msg = msg + video_quality + ' : '+  url+ '\n'
                    
            #Reply to the mentioned tweet with msg
            api.update_status(status= msg, in_reply_to_status_id = mention_tweet_id)
        except:
            error_msg = "Something went wrong :( Try again."
            api.update_status(status= error_msg, in_reply_to_status_id = mention_tweet_id)
            pass
        
print('Total Replies' , counter)
print('Last Tweet Id', last_tweet_id)

Total Replies 16
Last Tweet Id 1283338907602399232
